# Load data

In [1]:
import pandas as pd

df_dataset = pd.read_csv("preprocessed_dataset.csv",index_col=0).drop(columns=["1d_pct_price_var","10d_pct_price_var"])

# Train - test separation

In [2]:
df_train = df_dataset.iloc[:-900]
df_test = df_dataset.iloc[-900:]

x_train = df_train.drop(columns=["5d_pct_price_var","observation_date"])
y_train = df_train["5d_pct_price_var"]
x_test = df_test.drop(columns=["5d_pct_price_var","observation_date"])
y_test = df_test["5d_pct_price_var"]

# Model Hyperparameters Selection

In [3]:
# # !pip install lightgbm
# from lightgbm import LGBMRegressor
# from sklearn.model_selection import GridSearchCV
# import warnings
# warnings.filterwarnings('ignore', category=FutureWarning)


# estimator = LGBMRegressor(verbose=-1)
# param_grid = {
#     # "n_estimators": [5, 10, 50,100,500],
#     "n_estimators": [5,100,500],
#     "max_depth": [-1,5,10],
#     # "min_child_samples": [5, 10, 20, 50], 
#     "min_child_samples": [30,40,50,100,500], 
#     "learning_rate": [0.01, 0.02,0.05,0.2],
#     "lambda_l2":[0, 1, 2, 3]
# }

# grid_search = GridSearchCV(
#     estimator=estimator,
#     param_grid=param_grid,
#     scoring='r2',
#     cv=4,
#     verbose=1
# )

# grid_search.fit(x_train, y_train)

# print("LGBM")
# print("Best score:", grid_search.best_score_)
# print("Best parameters:", grid_search.best_params_)


In [4]:
# grid_search.score(x_test,y_test)

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer
import numpy as np
import optuna
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Custom scorer
def sign_accuracy(y_true, y_pred):
    return np.mean(np.sign(y_true) == np.sign(y_pred))

# Make scorer for sklearn
sign_acc_scorer = make_scorer(sign_accuracy, greater_is_better=True)

# Optuna objective
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 5, 505, step=50),
        "max_depth": trial.suggest_int("max_depth", 5, 10),
        "max_bin": trial.suggest_int("max_bin", 50, 100, step=10),
        "num_leaves": trial.suggest_int("num_leaves", 10, 35, step=5),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 5),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 0.9),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.9),
        "min_child_samples": trial.suggest_int("min_child_samples", 20, 100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 10),
        "device": "gpu",
        "verbose": -1,
        "random_state": 0
    }

    model = LGBMRegressor(**params)
    
    # Use cross-validation with custom scorer
    cv = KFold(n_splits=4, shuffle=True, random_state=0)
    scores = cross_val_score(model, x_train, y_train, cv=cv, scoring=sign_acc_scorer, n_jobs=-1)
    return np.mean(scores)

# Run Optuna Study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20, show_progress_bar=True)

print("\nLGBM Optuna Results")
print("Best Sign Accuracy score (CV):", study.best_value)
print("Best hyperparameters:", study.best_params)


# LGBM Optuna Results
# Best Sign Accuracy score (CV): 0.5783333333333334


[I 2025-07-04 11:24:05,049] A new study created in memory with name: no-name-2bc179c5-8b53-4ff3-b674-ac501e1deb35


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2025-07-04 11:24:13,523] Trial 0 finished with value: 0.5716666666666667 and parameters: {'n_estimators': 505, 'max_depth': 7, 'max_bin': 70, 'num_leaves': 30, 'bagging_freq': 2, 'bagging_fraction': 0.509333287708895, 'feature_fraction': 0.7458296641931426, 'min_child_samples': 25, 'learning_rate': 0.07169243980037304, 'lambda_l2': 9}. Best is trial 0 with value: 0.5716666666666667.
[I 2025-07-04 11:24:16,667] Trial 1 finished with value: 0.5625 and parameters: {'n_estimators': 105, 'max_depth': 9, 'max_bin': 50, 'num_leaves': 15, 'bagging_freq': 1, 'bagging_fraction': 0.7438694383831834, 'feature_fraction': 0.3551573282280933, 'min_child_samples': 67, 'learning_rate': 0.07062462067119979, 'lambda_l2': 2}. Best is trial 0 with value: 0.5716666666666667.
[I 2025-07-04 11:24:19,721] Trial 2 finished with value: 0.5495833333333333 and parameters: {'n_estimators': 155, 'max_depth': 7, 'max_bin': 50, 'num_leaves': 35, 'bagging_freq': 1, 'bagging_fraction': 0.5874480513791532, 'feature_fr

In [ ]:
from lightgbm import LGBMRegressor

params =  {'n_estimators': 355, 
           'max_depth': 7, 
           'max_bin': 50, 
           'num_leaves': 20, 
           'bagging_freq': 5, 
           'bagging_fraction': 0.7317950191499194, 
           'feature_fraction': 0.3238785519772445, 
           'min_child_samples': 77, 
           'learning_rate': 0.061232112293612306, 
           'lambda_l2': 2,
           'device':'gpu',
           'verbose': -1,
           'random_state': 0
           }
lgbm_reg = LGBMRegressor(**study.best_params, device="gpu", verbose=-1, random_state=0)
# lgbm_reg = LGBMRegressor(**params)
lgbm_reg.fit(x_train,y_train)

LGBMRegressor(bagging_fraction=0.7317950191499194, bagging_freq=5, device='gpu',
              feature_fraction=0.3238785519772445, lambda_l2=2,
              learning_rate=0.061232112293612306, max_bin=50, max_depth=7,
              min_child_samples=77, n_estimators=355, num_leaves=20,
              random_state=0, verbose=-1)

In [7]:
import numpy as np

def sign_accuracy(y_true, y_pred):
    return np.mean(np.sign(y_true) == np.sign(y_pred))

train_sign_accuracy = sign_accuracy(y_train,lgbm_reg.predict(x_train))
test_sign_accuracy = sign_accuracy(y_test,lgbm_reg.predict(x_test))

print("Train dataset performance: ",train_sign_accuracy)
print("Test dataset performance: ",test_sign_accuracy)

Train dataset performance:  0.8491666666666666
Test dataset performance:  0.4688888888888889


In [8]:
def sign_accuracy(y_true, y_pred):
    return np.mean(np.sign(y_true) == np.sign(y_pred))

# 1. Get numeric predictions (already continuous from LightGBM)
train_preds = lgbm_reg.predict(x_train)
test_preds = lgbm_reg.predict(x_test)

# 2. Calculate 90th percentile threshold of abs(preds) on train set
train_abs_preds = np.abs(train_preds)
threshold = np.percentile(train_abs_preds, 90)

# 3. Mask for predictions with abs value above threshold
train_mask = train_abs_preds > threshold
test_mask = np.abs(test_preds) > threshold

# 4. Calculate sign accuracy for only the confident predictions
train_sign_accuracy = sign_accuracy(y_train[train_mask], train_preds[train_mask])
test_sign_accuracy = sign_accuracy(y_test[test_mask], test_preds[test_mask])

print("Train Sign Accuracy (Top 10% most confident preds):", train_sign_accuracy)
print("Test Sign Accuracy (Top 10% most confident preds):", test_sign_accuracy)


Train Sign Accuracy (Top 10% most confident preds): 0.9916666666666667
Test Sign Accuracy (Top 10% most confident preds): 0.3333333333333333


In [ ]:
from scipy.stats import binom_test

# Example inputs
n_total = 900        # Total number of samples (adjust to your case)
n_correct = int(test_sign_accuracy * n_total)  # Number of correct sign predictions (52% of total)

# Perform one-sided binomial test (greater than 50%)
p_value = binom_test(n_correct, n_total, p=0.5, alternative='greater')

print(f"Sign Accuracy: {n_correct / n_total:.2%}")
print(f"P-value: {p_value:.5f}")

Sign Accuracy: 33.33%
P-value: 1.00000


C:\Users\volpe\AppData\Local\Temp\ipykernel_17964\65839734.py:8: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_value = binom_test(n_correct, n_total, p=0.5, alternative='greater')


: 